In [ ]:


import pandas as pd
from sklearn.tree import DecisionTreeClassifier

data = pd.read_csv("../../Data/known_behaviors_data.csv")
target_cols = ["Mortgage", "Pension", "Savings"]

X = data.drop(target_cols + ["customer_id"], axis=1, errors="ignore")

def get_thresholds(X, y, quantile=0.5):
    model = DecisionTreeClassifier(max_depth=2, random_state=42)
    model.fit(X, y)
    
    importance = pd.DataFrame({
        "feature": X.columns,
        "importance": model.feature_importances_
    }).sort_values("importance", ascending=False)
    
    top_features = importance.head(2)["feature"].tolist()
    
    thresholds = {
        "feature1": top_features[0],
        "threshold1": X[top_features[0]].quantile(quantile),
        "feature2": top_features[1],
        "threshold2": X[top_features[1]].quantile(quantile),
    }
    return thresholds

mortgage_thresh = get_thresholds(X, data["Mortgage"], quantile=0.35)
pension_thresh = get_thresholds(X, data["Pension"], quantile=0.35)
savings_thresh = get_thresholds(X, data["Savings"], quantile=0.35)

def generate_sql(thresholds, product_name):
    sql = f"""
    SELECT 
        customer_id
    FROM 
        clients
    WHERE 
        {thresholds['feature1']} > {thresholds['threshold1']}
        AND {thresholds['feature2']} > {thresholds['threshold2']}
    """
    return sql

mortgage_sql = generate_sql(mortgage_thresh, "Mortgage")
pension_sql = generate_sql(pension_thresh, "Pension")
savings_sql = generate_sql(savings_thresh, "Savings")

print("SQL для Mortgage:\n", mortgage_sql)
print("\nSQL для Pension:\n", pension_sql)
print("\nSQL для Savings:\n", savings_sql)

SQL для Mortgage:
 
    SELECT 
        customer_id
    FROM 
        clients
    WHERE 
        loan_accounts > 0.0
        AND members_in_household > 3.0
    

SQL для Pension:
 
    SELECT 
        customer_id
    FROM 
        clients
    WHERE 
        age > 35.0
        AND months_current_account > 15.0
    

SQL для Savings:
 
    SELECT 
        customer_id
    FROM 
        clients
    WHERE 
        income > 38486.5
        AND age > 35.0
    
